In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime as dt
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pd_data_base_path = '/content/drive/MyDrive/Preppin Data/Data/'

In [4]:
df = pd.read_csv(pd_data_base_path + 'DSB Customer Survery.csv')

In [5]:
df.head()

,Customer ID,Mobile App - Ease of Use,Mobile App - Ease of Access,Mobile App - Navigation,Mobile App - Likelihood to Recommend,Mobile App - Overall Rating,Online Interface - Ease of Use,Online Interface - Ease of Access,Online Interface - Navigation,Online Interface - Likelihood to Recommend,Online Interface - Overall Rating
0,535084,2,1,5,4,1,4,4,5,2,3
1,250892,3,5,4,4,2,5,5,2,4,3
2,544191,5,3,4,4,1,3,3,2,3,1
3,949343,2,5,4,3,1,1,4,3,5,1
4,915305,3,1,2,1,1,4,2,4,3,2


In [6]:
df = pd.melt(df, id_vars='Customer ID', var_name='Field', value_name='Rating')
df['Interface'] = df['Field'].str.extract(r'(.+) - ')
df['Field'] = df['Field'].str.split(' - ').str[1]
df['Mobile App'] = df.apply(lambda x: x['Rating'] if x['Interface']=='Mobile App' else 0, axis=1)
df['Online Interface'] = df.apply(lambda x: x['Rating'] if x['Interface']=='Online Interface' else 0, axis=1)
df.drop(columns=['Rating','Interface'], inplace=True)

In [7]:
df = df.groupby(['Customer ID', 'Field']).agg({'Mobile App':'sum','Online Interface':'sum'}).reset_index()
df.head()

,Customer ID,Field,Mobile App,Online Interface
0,101646,Ease of Access,5,4
1,101646,Ease of Use,3,2
2,101646,Likelihood to Recommend,4,4
3,101646,Navigation,2,3
4,101646,Overall Rating,5,2


In [8]:
df = df[df.Field != 'Overall Rating']
df.head()

,Customer ID,Field,Mobile App,Online Interface
0,101646,Ease of Access,5,4
1,101646,Ease of Use,3,2
2,101646,Likelihood to Recommend,4,4
3,101646,Navigation,2,3
5,101650,Ease of Access,4,5


In [9]:
df.shape

(3072, 4)

In [10]:
df = df.groupby('Customer ID').agg({'Mobile App':'mean','Online Interface':'mean'}).reset_index()
df.head()

,Customer ID,Mobile App,Online Interface
0,101646,3.50,3.25
1,101650,2.25,3.00
2,105088,3.50,4.25
3,109306,2.00,2.00
4,110719,3.00,3.50


In [11]:
df['Difference'] = df['Mobile App'] - df['Online Interface']
df.head()

,Customer ID,Mobile App,Online Interface,Difference
0,101646,3.50,3.25,0.25
1,101650,2.25,3.00,-0.75
2,105088,3.50,4.25,-0.75
3,109306,2.00,2.00,0.00
4,110719,3.00,3.50,-0.50


In [12]:
def categoriser(x):
  if x>=2:
    return 'Mobile App Superfan'
  elif x>=1:
    return 'Mobile App Fan'
  elif x>=0:
    return 'Neutral'
  elif x>=-1:
    return 'Online Interface Fan'
  elif x>=-2:
    return 'Online Interface Superfan'

df['Preference'] = df['Difference'].apply(categoriser)

In [13]:
df.head()

,Customer ID,Mobile App,Online Interface,Difference,Preference
0,101646,3.50,3.25,0.25,Neutral
1,101650,2.25,3.00,-0.75,Online Interface Fan
2,105088,3.50,4.25,-0.75,Online Interface Fan
3,109306,2.00,2.00,0.00,Neutral
4,110719,3.00,3.50,-0.50,Online Interface Fan


In [14]:
df = df.Preference.value_counts(normalize=True)*100

In [15]:
df = df.reset_index()

In [16]:
df.rename(columns={'index':'Preference','Preference':'% of Total'}, inplace=True)

In [17]:
df['% of Total'] = df['% of Total'].round(1)

In [18]:
df

,Preference,% of Total
0,Neutral,36.4
1,Online Interface Fan,33.5
2,Mobile App Fan,16.6
3,Online Interface Superfan,10.8
4,Mobile App Superfan,2.6
